In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
df = pd.read_table("train.tsv")
df.head()

,Unnamed: 0,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,0,39.4,4,85,70.0,2070,18.6,78,3,datsun b210 gx
1,3,18.0,6,171,97.0,2984,14.5,75,1,ford pinto
2,9,32.7,6,168,132.0,2910,11.4,80,3,datsun 280-zx
3,11,38.1,4,89,60.0,1968,18.8,80,3,toyota corolla tercel
4,13,28.0,4,98,80.0,2164,15.0,72,1,dodge colt (sw)


In [4]:
t=df["mpg"].values
x=df.drop(["mpg","car name"], axis=1).values

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.3, random_state=0)

In [6]:
import lightgbm as lgb

In [7]:
# LightGBM用のデータセットに入れる
lgb_train = lgb.Dataset(x_train, t_train)
lgb_test = lgb.Dataset(x_test, t_test)
# 評価基準を設定する 
params = {'metric' : 'rmse'}

In [8]:
# 訓練データから回帰モデルを作る
gbm = lgb.train(params, lgb_train)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000240 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 209
[LightGBM] [Info] Number of data points in the train set: 139, number of used features: 8
[LightGBM] [Info] Start training from score 23.437410
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [9]:
# テストデータを用いて予測精度を確認する
test_predicted = gbm.predict(x_test)
print(test_predicted)

[16.90583831 21.42317791 21.31761394 29.93239423 29.10224389 35.94666889
 18.78754973 15.05592978 19.80147601 19.70038816 18.99358541 27.45040753
 30.9743183  30.23121174 33.51505186 17.42979528 17.54936301 18.40997447
 15.19244579 13.60567468 17.16404558 19.15788707 37.66111409 12.70517992
 13.55497961 22.31880556 31.67674433 13.81970985 24.18691023 18.27899391
 34.885475   13.31209711 31.76379806 12.76735967 14.22111201 16.97842171
 22.37656181 27.29678935 16.97637129 26.106783   28.4008017  17.72381346
 23.66800896 32.89192062 16.06777252 13.55497961 14.82472213 20.62741816
 16.66092855 33.83677429 13.35605339 31.35885423 14.04708292 13.31644089
 20.82456318 29.59535976 31.48783069 12.76735967 36.67181504 27.62962221]


In [10]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
# モデル評価
# rmse : 平均二乗誤差の平方根
mse = mean_squared_error(t_test, test_predicted) # MSE(平均二乗誤差)の算出
rmse = np.sqrt(mse) # RSME = √MSEの算出
print('RMSE :',rmse)

#MAE
from sklearn.metrics import mean_absolute_error
mae=mean_absolute_error(t_test, test_predicted)
print("MAE",mae)

# r2 : 決定係数
r2 = r2_score(t_test,test_predicted)
print('R2 :',r2)

RMSE : 2.6478416934356734
MAE 1.9616828518531861
R2 : 0.898828664508117


In [11]:
df2 = pd.read_table("test.tsv")
df2.head()

,Unnamed: 0,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,1,8,350.0,180.0,4499,12.5,73,1,oldsmobile vista cruiser
1,2,4,156.0,105.0,2745,16.7,78,1,plymouth sapporo
2,4,6,156.0,108.0,2930,15.5,76,3,toyota mark ii
3,5,4,108.0,70.0,2245,16.9,82,3,toyota corolla
4,6,4,105.0,63.0,2125,14.7,82,1,plymouth horizon miser


In [21]:
x2_test=df2.drop(["car name"], axis=1)

In [22]:
x2_test.shape

(199, 8)

In [23]:
x_test.shape

(60, 8)

In [24]:
test2_predicted = gbm.predict(x2_test)

In [47]:
outdf = pd.Series(test2_predicted, index = df2["Unnamed: 0"].values )
outdf.head()

1    14.221112
2    23.672620
4    21.808780
5    36.139221
6    37.389825
dtype: float64

In [53]:
outdf.to_csv("submit.csv", index=True, header=False)